![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DeBertaForTokenClassification.ipynb)

## Import ONNX DeBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DeBertaForTokenClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBerta models trained/fine-tuned for token classification via `DeBertaForTokenClassification` or `TFDeBertaForTokenClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDeBertaForTokenClassification](https://huggingface.co/docs/transformers/model_doc/deberta#transformers.TFDebertaForTokenClassification)
- Some [example models](https://huggingface.co/models?filter=deberta&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [davanstrien/deberta-v3-base_fine_tuned_food_ner](https://huggingface.co/davanstrien/deberta-v3-base_fine_tuned_food_ner) model from HuggingFace as an example
- In addition to the DeBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DebertaV2Tokenizer
from optimum.onnxruntime import ORTModelForTokenClassification

MODEL_NAME = 'Angelakeke/RaTE-NER-Deberta'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/735M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

('onnx_models/Angelakeke/RaTE-NER-Deberta/tokenizer_config.json',
 'onnx_models/Angelakeke/RaTE-NER-Deberta/special_tokens_map.json',
 'onnx_models/Angelakeke/RaTE-NER-Deberta/spm.model',
 'onnx_models/Angelakeke/RaTE-NER-Deberta/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 721412
-rw-r--r-- 1 root root        23 Jun 12 00:56 added_tokens.json
-rw-r--r-- 1 root root      1399 Jun 12 00:56 config.json
-rw-r--r-- 1 root root 736235556 Jun 12 00:56 model.onnx
-rw-r--r-- 1 root root       970 Jun 12 00:56 special_tokens_map.json
-rw-r--r-- 1 root root   2464616 Jun 12 00:56 spm.model
-rw-r--r-- 1 root root      1314 Jun 12 00:56 tokenizer_config.json


- As you can see, we need to move `spm.model` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir -p {ONNX_MODEL}/assets && mv {ONNX_MODEL}/spm.model {ONNX_MODEL}/assets/

labels = [v for k, v in sorted(ort_model.config.id2label.items())]
with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/Angelakeke/RaTE-NER-Deberta:
total 719008
-rw-r--r-- 1 root root        23 Jun 12 00:56 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 12 00:56 assets
-rw-r--r-- 1 root root      1399 Jun 12 00:56 config.json
-rw-r--r-- 1 root root 736235556 Jun 12 00:56 model.onnx
-rw-r--r-- 1 root root       970 Jun 12 00:56 special_tokens_map.json
-rw-r--r-- 1 root root      1314 Jun 12 00:56 tokenizer_config.json

onnx_models/Angelakeke/RaTE-NER-Deberta/assets:
total 2412
-rw-r--r-- 1 root root     133 Jun 12 00:56 labels.txt
-rw-r--r-- 1 root root 2464616 Jun 12 00:56 spm.model


In [6]:
!cat {ONNX_MODEL}/assets/labels.txt

B-ABNORMALITY
I-ABNORMALITY
B-NON-ABNORMALITY
I-NON-ABNORMALITY
B-DISEASE
I-DISEASE
B-NON-DISEASE
I-NON-DISEASE
B-ANATOMY
I-ANATOMY
O

Voila! We have our `spm.model` and `labels.txt` inside assets directory

## Import and Save DeBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 37.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DeBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [9]:
from sparknlp.annotator import DeBertaForTokenClassification

tokenClassifier = DeBertaForTokenClassification.loadSavedModel(
      ONNX_MODEL,
      spark
      )\
    .setInputCols(["document",'token'])\
    .setOutputCol("ner")\
    .setCaseSensitive(True)\
    .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DeBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 721512
-rw-r--r-- 1 root root 736348035 Jun 12 00:59 deberta_classification_onnx
-rw-r--r-- 1 root root   2464616 Jun 12 00:59 deberta_spp
drwxr-xr-x 3 root root      4096 Jun 12 00:59 fields
drwxr-xr-x 2 root root      4096 Jun 12 00:59 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForTokenClassification model 😊

In [13]:
tokenClassifier_loaded = DeBertaForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
tokenClassifier_loaded.getClasses()

['I-ANATOMY',
 'B-ABNORMALITY',
 'B-DISEASE',
 'B-ANATOMY',
 'I-NON-ABNORMALITY',
 'I-NON-DISEASE',
 'I-ABNORMALITY',
 'B-NON-DISEASE',
 'B-NON-ABNORMALITY',
 'I-DISEASE',
 'O']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [18]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example = spark.createDataFrame([["The MRI showed a lesion in the left temporal lobe, but no evidence of stroke or tumor was found."],]).toDF("text")
result = pipeline.fit(example).transform(example)

result.select("text", "ner.result").show(truncate=False)
result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                            |result                                                                                                                                                                         |
+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|The MRI showed a lesion in the left temporal lobe, but no evidence of stroke or tumor was found.|[O, B-ANATOMY, I-ANATOMY, O, O, O, O, O, O, O, O, O, B-ABNORMALITY, I-ABNORMA

That's it! You can now go wild and use hundreds of `DeBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
